# Intent Classification with ThirdAI's Universal Deep Transformer
This notebook shows how to build an intent classification model with ThirdAI's Universal Deep Transformer (UDT) model, our all-purpose classifier for tabular datasets. In this demo, we will train and evaluate the model on the CLINC 150 dataset, but you can easily replace this with your own dataset.

You can immediately run a version of this notebook in your browser on Google Colab at the following link:

https://githubtocolab.com/ThirdAILabs/Demos/blob/main/IntentClassification.ipynb

This notebook uses an activation key that will only work with this demo. If you want to try us out on your own dataset, you can obtain a free trial license at the following link: https://www.thirdai.com/try-bolt/

In [ ]:
!pip3 install thirdai --upgrade

# This activates the ThirdAI package with a key that is only good for this demo
import thirdai
thirdai.activate("PRCH-AAVV-3ART-3WY9-PM9H-KULT-PRW3-C399")

# Dataset Download
We will use the demos module in the thirdai package to download the CLINC 150 dataset. You can replace this step and the next step with a download method and a UDT initialization step that is specific to your dataset.

In [ ]:
from thirdai.demos import download_clinc_dataset

train_filename, test_filename, inference_batch = download_clinc_dataset()

# UDT Initialization
We can now create a UDT model by passing in the types of each column in the dataset and the target column we want to be able to predict.

In [ ]:
from thirdai import bolt

model = bolt.UniversalDeepTransformer(
    data_types={
        "text": bolt.types.text(),
        "category": bolt.types.categorical(),
    },
    target="category",
    n_target_classes=151,
)

# Training
We can now train our UDT model with just one line! Feel free to customize the number of epochs and the learning rate; we have chosen values that give good convergence.

In [ ]:
model.train(train_filename, epochs=5, learning_rate=0.01, metrics=["categorical_accuracy"])

# Evaluation
Evaluating the performance of the UDT model is also just one line!

In [ ]:
model.evaluate(test_filename, metrics=["categorical_accuracy"]);

# Saving and Loading
Saving and loading a trained UDT model to disk is also extremely straight forward.

In [ ]:
save_location = "intent_classification.model"

# Saving
model.save(save_location)

# Loading
model = bolt.UniversalDeepTransformer.load(save_location)

# Testing Predictions
The evaluation method is great for testing, but it requires labels, which don't exist in a production setting. We also have a predict method that can take in an in-memory batch of rows or a single row (without the target column), allowing easy integration into production pipelines.

In [ ]:
import numpy as np

print("Inference batch:", inference_batch, "\n")

prediction = model.predict(inference_batch[0])
class_name = model.class_name(np.argmax(prediction))
print("Input:", inference_batch[0], "Prediction:", class_name, "\n")

prediction_batch = model.predict_batch(inference_batch)
class_names = [
    model.class_name(class_id) for class_id in np.argmax(prediction_batch, axis=1)
]
print("Batch Prediction Results")
for input_sample, class_name in zip(inference_batch, class_names):
    print("Input:", input_sample, "Prediction:", class_name)